In [ ]:
!pip install netCDF4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
import string

In [ ]:
def createImage(file_name, global_vmin, global_vmax,type):
  nc = netCDF4.Dataset(file_name)
  time = nc.variables['time'][:]
  lon = nc.variables['lon'][:]
  lat = nc.variables['lat'][:]
  var = nc.variables[type][:]
  arr=[]

  for time_index in range(len(time)):
      tas_at_time = var[time_index, :, :]
      plt.figure(figsize=(10, 10))
      plt.imshow(tas_at_time, origin='lower', extent=[lat.min(), lat.max(), lon.min(), lon.max()], aspect='auto', cmap='coolwarm', vmin=global_vmin, vmax=global_vmax)
      info = 'Temperature at Surface (tas)' if type else 'Precipitation at Surface (pr)'
      plt.title(f'{info} at Time Step {time_index+1} for {file_name}')
      plt.xlabel('Latitude')
      plt.ylabel('Longitude')
      plt.colorbar(label= type)
      arr.append(plt.gcf())
      plt.close()
  print(arr)
  return arr

In [ ]:
#the bounds are already found by analzing how much data each source has and we get that to be
#1992-2016, so it is a pretty simple fix
def find_names(start_file_name):
  arr=[]
  currName=start_file_name
  currYr=1992
  maxYr=2016
  ran=maxYr-currYr
  for i in range(ran+1):
    arr.append(currName)
    currName = currName.replace(str(currYr), str(currYr+1))
    currYr+=1
  return np.array(arr)

In [ ]:
#In order to scale all the heatmaps of the data the same you have to get the variable min
#and max by looping through it all which is basically what this code segent does
def calc_bounds(files, type):
  global_var_min=10e9
  global_var_max=10e-9
  for i in files:
    nc = netCDF4.Dataset(i)
    time = nc.variables['time'][:]
    for time_index in range(len(time)):
      var = nc.variables[type][:]
      global_var_min=min(global_var_min, var.min())
      global_var_max=max(global_var_max, var.max())
  return global_var_min,global_var_max

In [ ]:
def return_final_plots(starter):
  type='tas'
  if 'precip' in starter:
    type='pr'
  listOfFiles=find_names(starter)
  currmin,currmax=calc_bounds(listOfFiles, type)
  plots=[]
  for i in listOfFiles:

        plot = createImage(i, currmin, currmax, type)
        plots.append(plot)

  print(plots)
  return plots

In [ ]:
currPrepPlots = return_final_plots("/content/drive/MyDrive/dataset-insitu-gridded-observations-global-and-regional-077c7a7b-663b-4bfd-9628-d732561c7baa/CRU_total_precipitation_mon_0.5x0.5_global_1992_v4.03.nc")
currTempPlots = return_final_plots( "/content/drive/MyDrive/dataset-insitu-gridded-observations-global-and-regional-077c7a7b-663b-4bfd-9628-d732561c7baa/CRU_mean_temperature_mon_0.5x0.5_global_1992_v4.03.nc")
for i in currPrepPlots:
  for j in i:
    j.show()
for i in currTempPlots:
  for j in i:
    j.show()


Starter File= "/content/drive/MyDrive/dataset-insitu-gridded-observations-global-and-regional-077c7a7b-663b-4bfd-9628-d732561c7baa/CRU_total_precipitation_mon_0.5x0.5_global_1992_v4.03.nc"

Starter File= "/content/drive/MyDrive/dataset-insitu-gridded-observations-global-and-regional-077c7a7b-663b-4bfd-9628-d732561c7baa/CRU_mean_temperature_mon_0.5x0.5_global_1992_v4.03.nc"